**<font color = black size=6>实验六:决策树</font>**

**<font color = blue size=4>第一部分:函数介绍</font>**

In [2]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

1)Counter类的使用

In [3]:
x=np.array([[0,133,1],[0,132,0],[0,133,0]])
#使用Counter类对数组第2列进行遍历
counter = Counter(x[:,1])
#第2列中有1个132和2个133，输出该counter对象可以统计这列的数值情况，便于之后的统计
print(counter)
#因为第2列中没有为0的值，所以返回0
print(counter[0])
#因为第2列中有2个133，所以返回2
print(counter[133])
#一般的字典操作方法都能在该类中使用，例如可以通过values函数返回该列的非重复值的个数，方便对某列的非重复值的个数进行查看
print(counter.values())
#可以输出所有非重复值
print(list(counter))

Counter({133: 2, 132: 1})
0
2
dict_values([2, 1])
[133, 132]


2)使用numpy中的unique实现非重复值的提取

In [4]:
x=np.array([[0,133,1],[0,132,0],[0,133,0]])
a=np.unique(x[:])
a1=np.unique(x[:,1])
print(a,a1)

[  0   1 132 133] [132 133]


**<font color = blue size=4>第二部分:实验任务 Part I--编写函数</font>**

<span style="color:purple">该数据集(train_titanic.csv)为分类数据集，为泰坦尼克号的部分乘客信息以及最后是否生还。包括了四个属性值以及一个标记属性(即为Survived类型,代表是否生还),属性信息分别为Sex(性别)，sibsp(堂兄弟妹个数)，Parch(父母与小孩的个数)，Pclass(乘客等级)。  
其中该数据集无缺失值和异常值，且所有连续变量已自动转换为离散变量,标记(Survived类型)也自动转变为离散变量0和1，所以你无需进行数据预处理，可以直接使用该数据集。</span>

In [5]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import math
from collections import Counter

<span style="color:purple">1) 使用pandas库将训练数据集'train_titanic.csv'载入到Dataframe对象中</span>

In [6]:
# 加载数据集
data = pd.read_csv('train_titanic.csv')
print(data)

      Sex  sibsp  Parch  Pclass  Survived
0       0      1      0       3         0
1       1      1      0       1         1
2       1      0      0       3         1
3       1      1      0       1         1
4       0      0      0       3         0
...   ...    ...    ...     ...       ...
1004    0      0      0       3         0
1005    1      0      0       1         0
1006    0      0      0       3         0
1007    0      0      0       3         0
1008    0      1      1       3         0

[1009 rows x 5 columns]


<span style="color:purple">2) 编写函数，给定任何标记数组计算其信息熵  
    输入：标记数组  
    输出：该数组对应的信息熵  
    计算信息熵公式:
某数组包含K个不同的取值，样本为第k(k=1,2,...,K)个值的数量所占比例为p_k,则其信息熵为$$Ent=-\sum_{k=1}^K p_k log_2 p_k$$</span>
    
    
<span style="color:purple">例:  
    输入:[[0],[1]]   
    输出:(-$\frac{1}{2}$ $log_2$ $\frac{1}{2}$)+(-$\frac{1}{2}$ $log_2$ $\frac{1}{2}$)=1</span>

In [7]:
1.
#定义计算信息熵函数
def calculate_entropy(labels):
    label_counts = Counter(labels)
    entropy = 0
    for count in label_counts.values():
        probability = count / len(labels)
        entropy -= probability * math.log2(probability)
    return entropy
#调用函数，求解最后一列列的信息熵
features = data.iloc[:, :-1].values  # 特征值
labels= data.iloc[:, -1].values      # 标签值
print("信息熵是：",calculate_entropy(labels)) # 调用函数计算信息熵

信息熵是： 0.7709131174627031


<span style="color:purple">3) 编写函数，函数功能为将所给的数据集按照指定维度dimension进行划分为若干个不同的数据集  
    【输入】：属性集合，标记集合，维度索引  
    【输出】：划分后所得到的子树属性集合，子树标记集合</span>

<span style="color:purple">例子:  
    【输入】:feature(属性值集合):[[0,0,0],[0,0,1],[1,0,2]]  
    label(标记集合):[[0],[1],[2]]  
    划分维度:0</span>  
    
<span style="color:purple">【输出】:[[0,0,0],[0,0,1]]和[[1,0,2]]  
    [[0],[1]]和[[2]]  
    tips:即将feature按其第0维度进行划分，由于feature的0维有0和1两个不同的数值，所以feature划分为[[0,0,0],[0,0,1]]和[[1,0,2]]，label划分为[[0],[1]]和[[2]]</span>

In [8]:
2.
#定义划分不同数据集函数
def split_dataset(features, labels, dimension):
    unique_values = np.unique([row[dimension] for row in features])
    split_subsets = []
    for value in unique_values:
        subset = [[], []]  # 第一个列表存放属性，第二个列表存放标记
        for i, row in enumerate(features):
            if row[dimension] == value:
                subset[0].append(row)
                subset[1].append(labels[i])
        split_subsets.append(subset)
    return split_subsets

<span style="color:purple">4) 编写函数，函数功能为进行【一次】决策树的结点划分，遍历找出该属性集合中信息增益(使用ID3算法中的公式计算)【最大】的属性  
    输入：属性集合，标记集合  
    输出：该次划分的最佳信息增益，最佳划分维度  
    计算信息增益公式:  
    某数据集D有若干属性值以及对应的标记值，其总样本大小为|D|,这里取其中一个属性类型feature,该特征包含V个不同的取值，属性值为第v(v=1,2,...,V)个值的数量为|$D^v$|$(\sum_{v=1}^V |D^v|=|D|)$,则该属性值对应的信息增益为为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^V \frac{|D^v|}{|D|} Ent(D^v)$$
所以该函数中你需要计算出每个属性值的信息增益并输出，然后返回最大的信息增益并返回该特征的维数以及最大的信息增益值</span>

In [9]:
3.
#定义信息增益最大的决策划分（ID3算法）
def find_best_split_id3(features, labels):
    base_entropy = calculate_entropy(labels)
    best_gain = 0
    best_dimension = -1
    num_features = len(features[0])
    #遍历
    for i in range(num_features):
        subsets = split_dataset(features, labels, i)
        new_entropy = 0
        for subset in subsets:
            subset_entropy = calculate_entropy(subset[1])
            new_entropy += (len(subset[1]) / len(labels)) * subset_entropy
        info_gain = base_entropy - new_entropy
        if info_gain > best_gain:
            best_gain = info_gain
            best_dimension = i
    
    return best_gain, best_dimension

<span style="color:purple">5) 编写函数，函数功能为进行【一次】决策树的结点划分，遍历找出该属性集合中信息增益率(使用C4.5算法中的公式计算)【最大】的属性  
    输入：属性集合，标记集合  
    输出：该次划分的信息增益率，最佳维度  
    计算信息增益率公式:  
    某数据集D有若干属性值以及对应的标记值，其总样本大小为|D|,这里取其中一个属性类型feature,该属性包含V个不同的取值，属性值为第v(v=1,2,...,V)个值的数量为|$D^v$|$(\sum_{v=1}^V|D^v|=|D|)$,该属性本身的信息熵为Ent(feature),则该属性值对应的信息增益率为$$Gain\_ratio(D,feature)=\frac{Gain(D,feature)}{Ent(feature)}$$
所以该函数中你需要输出每个特征的信息增益率，之后返回最大的信息增益率对应的属性维数以及最大的信息增益率</span>

In [10]:
4.
#定义信息增益率最大决策划分（C4.5算法）
#需要结合前面的ID3算法和计算信息熵的函数相除
def find_best_split_c45(features, labels):
    base_entropy = calculate_entropy(labels)
    best_gain_ratio = 0
    best_dimension = -1
    num_features = len(features[0])
    
    for i in range(num_features):
        subsets = split_dataset(features, labels, i)
        new_entropy = 0
        split_info = 0
        for subset in subsets:
            subset_entropy = calculate_entropy(subset[1])
            proportion = len(subset[1]) / len(labels)
            new_entropy += proportion * subset_entropy
            split_info -= proportion * math.log2(proportion)
        info_gain = base_entropy - new_entropy
        if split_info != 0:
            gain_ratio = info_gain / split_info
            if gain_ratio > best_gain_ratio:
                best_gain_ratio = gain_ratio
                best_dimension = i
    return best_gain_ratio, best_dimension

<span style="color:purple">6) 编写函数，进行【一次】决策树的结点划分，遍历找出该属性集合中基尼系数(使用CART算法中的公式计算)最小的属性以及最佳的划分值  
    输入：属性集合，标记集合  
    输出：该次划分的最佳的基尼系数，最佳维度，最佳划分值  
    计算基尼系数公式:  
    某数据集D有若干属性值以及对应的标记值，其总样本大小为|D|,该集合中样本标记类别总共有K类，第k类样本所占比例为$p_k$(k=1,2,..,K)则该数据集对应的基尼系数为$$Gini(D)=1-\sum_{k=1}^K{p_k}^2$$  
    而取其中一个属性类型feature，选定该类型的一个值value，根据feature这个属性是否为value将数据集分为两个子集$D_1$和$D_2$,则该属性feature对应的基尼系数为$$Gini\_index(D,feature)=\frac{|D_1|}{|D|}Gini(D_1)+\frac{|D_2|}{|D|}Gini(D_2)$$
所以该函数中你需要首先找出每列中的非重复值，之后根据该列的非重复值进行分类，计算出该列中以该值进行分类的基尼系数，计算出每个属性的每个非重复值的基尼系数，之后找到最小的基尼系数对应的属性维数以及对应的分类值</span>

In [11]:
5.
#利用基尼系数进行划分：
def find_best_split_cart(features, labels):
    best_gini_index = float('inf')
    best_dimension = -1
    best_split_value = None
    num_features = len(features[0])
    
    for i in range(num_features):
        unique_values = np.unique([row[i] for row in features])
        for value in unique_values:
            subsets = [[], []]  # 分别存储两个子集的标记
            for j, row in enumerate(features):
                if row[i] == value:
                    subsets[0].append(labels[j])
                else:
                    subsets[1].append(labels[j])
            gini_index = sum(
                [(len(subset) / len(labels)) * (1 - sum([((subset.count(k) / len(subset)) ** 2) for k in np.unique(subset)])) 
                 for subset in subsets])
            if gini_index < best_gini_index:
                best_gini_index = gini_index
                best_dimension = i
                best_split_value = value
    
    return best_gini_index, best_dimension, best_split_value

<span style="color:purple">7) 应用之前你在第4、5、6个部分编写的三个函数，在训练数据集'train_titanic.csv'上依次使用这些函数进行【一次】结点划分，并输出对应的最佳属性维数以及相应的信息增益值/信息增益率/(基尼系数和分类值)</span>

In [12]:
7.
#实现一次划分
features = data.iloc[:, :-1].values  # 特征值
labels= data.iloc[:, -1].values      # 标签值
print("信息熵是：",calculate_entropy(labels)) # 调用函数计算信息熵




#使用ID3算法进行信息划分：
best_gain, best_dimension=find_best_split_id3(features, labels)       #ID3算法
print("ID3:")
print("best_gain:",best_gain)
print("best_dimension:",best_dimension)


#使用C4.5算法进行信息划分：
best_gain_ratio, best_dimension2=find_best_split_c45(features, labels)
print("C4.5:")
print("best_gain:",best_gain)
print("best_dimension:",best_dimension)


#使用gini函数进行信息划分：
best_gini_index, best_dimension3, best_split=find_best_split_cart(features, labels)
print("GINI:")
print("best_gini_index:",best_gini_index)
print("best_dimension:",best_dimension)
print("best_split:",best_split)

信息熵是： 0.7709131174627031
ID3:
best_gain: 0.10750711887455178
best_dimension: 0
C4.5:
best_gain: 0.10750711887455178
best_dimension: 0
GINI:
best_gini_index: 0.2964915724641573
best_dimension: 0
best_split: 0
